In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import datetime
import math
import random
import os

In [4]:
file_path = "./normal.csv"
df = pd.read_csv(file_path)
norvalue = df['value'].values

file_path = "./traindata.csv"
df = pd.read_csv(file_path)
convalue = df['value'].values

In [5]:
class Pid():
    def __init__(self, exp_val, con_val, kp, ki, kd):
        self.KP = kp
        self.KI = ki
        self.KD = kd
        self.exp_val = exp_val
        self.now_val = con_val
        self.sum_err = 0
        self.now_err = 0
        self.last_err = 0

    def cmd_pid(self):
        self.last_err = self.now_err
        self.now_err = self.exp_val - self.now_val
        self.sum_err += self.now_err
        # 这一块是严格按照公式来写的
        self.now_val = self.KP * (self.exp_val - self.now_val) \
                        + self.KI * self.sum_err + self.KD * (self.now_err - self.last_err)
        return self.now_val

In [6]:
# ----------------------PSO参数设置---------------------------------
class PSO():
    def __init__(self, pN, dim, max_iter):
        self.w = 1
        self.c1 = 2
        self.c2 = 2
        self.r1 = 0.6
        self.r2 = 0.3
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.max_vel = 0.05
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置和速度
        self.V = np.zeros((self.pN, self.dim))
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置和全局最佳位置
        self.gbest = np.zeros((1, self.dim))
        self.p_fit = np.zeros(self.pN)  # 每个个体的历史最佳适应值
        self.fit = 1e10  # 全局最佳适应值

    # ---------------------目标函数-----------------------------
    def function(self, kpidi):
        lastvalue = []   #存储最后pid迭代的值
        for j in range(0,2501):   #一次控制2501个数据(即0.2s)，共用一个pid控制器
            my_Pid = Pid(norvalue[j], convalue[j], kpidi[0], kpidi[1], kpidi[2])   # nor正常  con异常  kp, ki, kd
            nowvalue = 0
            for i in range(1,3):  #每个数据迭代二次
                nowvalue = my_Pid.cmd_pid()  

        lastvalue.append(nowvalue)
        return np.linalg.norm(norvalue-lastvalue)

    # ---------------------初始化种群----------------------------------
    def init_Population(self):
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = random.uniform(0, 1)
                self.V[i][j] = random.uniform(0, 1)
            self.pbest[i] = self.X[i]
            tmp = self.function(self.X[i])
            self.p_fit[i] = tmp
            if tmp < self.fit:
                self.fit = tmp
                self.gbest = self.X[i]

                # ----------------------更新粒子位置----------------------------------

    def iterator(self):
        fitness = []
        for t in range(self.max_iter):
            
            for i in range(self.pN):  # 更新gbest\pbest
                temp = self.function(self.X[i])
                if temp < self.p_fit[i]:  # 更新个体最优
                    self.p_fit[i] = temp
                    self.pbest[i] = self.X[i]
                    if self.p_fit[i] < self.fit:  # 更新全局最优
                        self.gbest = self.X[i]
                        self.fit = self.p_fit[i]
                        print(self.gbest, end=" ")
                        print(self.fit)  # 输出最优值
            for i in range(self.pN):
#                 self.V[i] = self.w * self.V[i] + self.c1 * self.r1 * (self.pbest[i] - self.X[i]) + \
#                             self.c2 * self.r2 * (self.gbest - self.X[i])
                self.V[i] = self.w * self.V[i] + self.c1 * random.random() * (self.pbest[i] - self.X[i]) + \
                            self.c2 * random.random() * (self.gbest - self.X[i])     
                
                for k in range(0,self.dim):
                    if self.V[i][k]>self.max_vel:
                        self.V[i][k]=self.max_vel
                    if self.V[i][k]<-self.max_vel:
                        self.V[i][k]=-self.max_vel
              
                    
                self.X[i] = self.X[i] + self.V[i]
                

            fitness.append(self.fit)
#             print(self.X[0], end=" ")
#             print(self.fit)  # 输出最优值
        return fitness
 

In [7]:
my_pso = PSO(pN=30, dim=3, max_iter=100)
my_pso.init_Population()
fitness = my_pso.iterator()

[ 0.11471833 -0.01487683  0.12009754] 837.9941062176435
[0.0643433  0.49830651 0.09473451] 816.859796524068
[ 0.06471833 -0.06487683  0.07009754] 755.8842239805923
[0.02251979 0.56438134 0.0568013 ] 719.5041585378294
[0.01961229 0.44830651 0.04473451] 692.8776890742325
[0.04425686 0.28816374 0.03489848] 692.2622674864289
[ 0.13838052  0.26788364 -0.00983685] 692.1821540281775
[ 0.12965213  0.26452971 -0.0057772 ] 692.1797144136266
[0.09367021 0.23749906 0.01312016] 692.1796204794957
[0.11257014 0.20711179 0.00347529] 692.1794812926994
